# Final Project

## 4/20/2025

- Team Members
    - Brittney Bedrossian
    - Breenda Shah
    - Jeb Besecker
    - Kevin Dennin
    - Kohry Long

### This serves as an initial exploration and data pre processing module. 

In [ ]:
#hello world

# Imports 

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mno
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import graphviz
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Changelog Intialization

In [ ]:
# Initilize changelog
changelog = []

# Data Load

In [ ]:
# Load the dataset
df = pd.read_csv("../Data/loan_data.csv")
df.head()

In [1]:
df.info()


NameError: name 'df' is not defined

In [ ]:
# Intial Stats of numerical variables
df.describe()

**Initial observations of the numerical variables** - The dataset contains 45,000 records and provides insights into individuals' demographics, income, employment experience, and loan-related details. The average age of individuals is approximately 27.76 years, with a minimum of 20 and a maximum of 144 years, suggesting a likely data outlier at the upper end. Similarly, the person_income variable has a mean of about $80,319, but a maximum value of over $7.2 million indicates the presence of extreme outliers. Employment experience (person_emp_exp) averages around 5.41 years, though the maximum value of 125 years is likely unrealistic and could also be an error. Loan amounts range from $500 to $35,000, with a mean of approximately $9,583, while interest rates vary from 5.42% to 20%, averaging around 11%. The loan_percent_income, which represents the loan amount as a proportion of income, has a mean of 0.14. Credit history length and credit scores also show considerable variation, with the average credit score being 632 and ranging from 390 to 850. Lastly, the loan_status variable indicates that around 22% of individuals in the dataset received loan approval. Overall, the data shows strong variability, and several features may require cleaning or normalization due to the presence of clear outliers.

In [ ]:
# Intial Stats of categorical variables
df.describe(include = object)

**Initial observation of categorical variables** - the majority of the population is male, accounting for 24,841 entries. Education levels are diverse, with five unique categories, and the most common qualification is a Bachelor’s degree, held by 13,399 individuals. When it comes to housing status, renting is the most frequent form of home ownership, with 23,443 individuals reporting it. The dataset also records loan intentions, with six distinct purposes, of which education-related loans are the most common, making up 9,153 entries. Lastly, the variable indicating whether individuals had previous loan defaults shows an almost even split, with 22,858 people having a history of loan defaults.

In [ ]:
# Explore MissingNo
mno.matrix(df)

# Data Exploration and Categorical Analysis

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns:", categorical_columns)

In [ ]:
# Univariate analysis of categorical variables
for col in categorical_columns + ['loan_status']:
    fig = plt.figure(figsize=(8, 4))
    df[col].value_counts(normalize=True).plot(kind='bar')
    plt.title(f"Distribution of {col}")
    plt.show()

# Notes on Categorical Analysis:
Firstly, there are no spelling errors present in the dataset. Secondly, given that the "Own" and "Other" categories constitute a small proportion of the person_home_ownership variable, it may be appropriate to consolidate them into a single "Others" category for more effective analysis. Lastly, the overall distribution of the remaining categorical variables appears well-balanced. However, an exception is observed in the person_education variable, where the "Doctorate" group represents a notably small subset of the population - consider dropping this all together. 

# Numerical Analysis

In [ ]:
# Univariate analysis of numerical variables
numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist()
print("Numerical columns:", numerical_columns)
for col in numerical_columns:
    fig = plt.figure(figsize=(8, 4))
    sns.histplot(df[col], kde=True)
    plt.title(f"Distribution of {col}")
    plt.show()

In [ ]:
# Checking for skewness in each of the numerical variables
numericalColumns = df.select_dtypes(include=[ 'int64'])
numericalColumns[:10]              

#Checking for skewness in the numerical columns.
skewness = df[numericalColumns.columns].skew()
print(skewness)

# Notes on Numerical variables
The skewness results for the numerical variables indicate that several distributions are significantly non-normal, with a strong tendency toward right skewness. Notably, person_income shows an extreme right skew (skewness = 34.14), suggesting the presence of a few individuals with very high incomes that distort the overall distribution. Similarly, person_age and person_emp_exp both exhibit strong right skewness (2.55 and 2.59, respectively), likely influenced by outliers such as individuals with unusually high age or employment experience values as mentioned earlier in summary stats. The loan_amnt and cb_person_cred_hist_length variables also display moderate right skewness, indicating that most applicants request lower loan amounts and have relatively short credit histories. In contrast, credit_score shows a moderate left skew (-0.61), suggesting a concentration of higher credit scores with fewer individuals having poor credit. Lastly, the loan_status variable is moderately right-skewed (1.34), reflecting an imbalance in loan approvals versus rejections, which may require attention during classification tasks. 

In [ ]:
# Correlation matrix
plt.figure(figsize=(12, 8))
correlation_matrix = df.select_dtypes(include=['float64', 'int64']).corr()
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


# Notes on Correlation Matrix
Strong positive correlations: person_age and person_emp_exp (0.95): Older individuals tend to have more work experience, person_age and cb_person_cred_hist_length (0.86): Older applicants generally have longer credit histories, and person_emp_exp and cb_person_cred_hist_length (0.82): More experienced individuals also tend to have longer credit histories.

Moderate correlations:loan_amnt and loan_percent_income (0.59): Larger loan amounts typically make up a higher percentage of a person’s income, loan_status and loan_percent_income (0.38): Applicants with a higher loan-to-income ratio are more likely to be rejected, and loan_status and loan_int_rate (0.33): Higher interest rates are moderately associated with loan rejection.

Weak or negligible correlations:loan_status has weak correlation with most features, including person_age, person_income, person_emp_exp, and credit_score and person_income shows very little correlation with other variables, except a slight negative correlation with loan_percent_income (-0.23).

# Identifying Numerical Variable Multicolinearity 

In [ ]:
num_cols = ['person_income', 'person_emp_exp', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length','credit_score']

# Subset the numerical columns
X = df[num_cols].copy()

# Add constant term for VIF calculation (intercept)
X_const = add_constant(X)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]

# Drop constant term (intercept) from results for clarity
vif_data = vif_data[vif_data["Feature"] != "const"]

print(vif_data)

#Pair plot of numerical columns
numerical_vars = ['person_income', 'person_emp_exp', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length','credit_score']
sns.pairplot(df[numerical_vars])
sns.set_style("whitegrid")
plt.show()

**Notes on Multicolinearity of Numerical variables**
The Variance Inflation Factor (VIF) analysis indicates that there is no significant multicollinearity among the selected numerical features in the dataset. All VIF values fall well below the common threshold of 5, suggesting that none of the variables are excessively correlated with one another. Specifically, variables such as person_income, loan_int_rate, and credit_score exhibit very low VIF values, confirming their independence from the other predictors. While person_emp_exp and cb_person_cred_hist_length show moderately higher VIFs around 3, these values are still within acceptable limits and do not pose a concern. Overall, the dataset's numerical features are suitable for use  without the risk of multicollinearity distorting the results. Additionally, the par plot between the numerical columns further suggest no significant relationship amongst the columns. 

# Data Cleaning & Preprocessing

In [ ]:
# Copy original df to df_cleaned
df_cleaned = df.copy()

In [ ]:
# Remove unrealistic ages
original_row_count = df_cleaned.shape[0]
df_cleaned = df_cleaned[df_cleaned['person_age'] <= 125]
removed_count = original_row_count - df_cleaned.shape[0]


changelog.append({
    'column': 'age',
    'change': f'Removed {removed_count} row(s) where age > 125',
    'rationale': 'Oldest verified human lived to be 122; values above 125 are likely errors or outliers.',
})


In [ ]:
# Drop loan_status column
df_cleaned.drop(columns=['loan_status'], inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)
numerical_columns = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()
changelog.append({
    'column': 'loan_status',
    'change': 'Dropped loan_status column',
    'rationale': 'Target variable for classification; not needed in cleaned dataset.',
})


In [ ]:
# Cap outliers before log-transform
df_cleaned['person_income'] = np.where(df_cleaned['person_income'] > 3_000_000, 3_000_000, df_cleaned['person_income'])


# Show numerical columns after capping
print("Numerical columns after capping:", numerical_columns)


# Apply log1p to preserve 0s and small values
df_cleaned['log_income'] = np.log1p(df_cleaned['person_income'])
changelog.append({
    'column': 'person_income',
    'change': 'Capped person_income at $3,000,000 and applied log1p transformation',
    'rationale': 'Handled extreme right skew and outliers in income distribution; log1p applied to normalize values and removed person_income for modeling.',
})


plt.figure(figsize=(8, 4))
sns.histplot(df_cleaned['person_income'], bins=50, kde=True)
plt.title("Income Before Log")
plt.show()

plt.figure(figsize=(8, 4))
sns.histplot(df_cleaned['log_income'], bins=50, kde=True)
plt.title("Income After Log")
plt.show()


In [ ]:
# Drop person_income after log-transform
df_cleaned.drop(columns=['person_income'], inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)
numerical_columns = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()


In [ ]:
# Apply log1p to preserve 0s and small values
df_cleaned['log_loan_amt'] = np.log1p(df_cleaned['loan_amnt'])
changelog.append({
    'column': 'loan_amnt',
    'change': 'Applied log1p transformation to loan_amount',
    'rationale': 'Handled extreme right skew and outliers in loan amount distribution; log1p applied to normalize values.',
})

plt.figure(figsize=(8, 4))
sns.histplot(df_cleaned['loan_amnt'], bins=50, kde=True)
plt.title("Loan Before Log")
plt.show()

plt.figure(figsize=(8, 4))
sns.histplot(df_cleaned['log_loan_amt'], bins=50, kde=True)
plt.title("Loan After Log")
plt.show()

In [ ]:
# Drop loan_amnt after log-transform
df_cleaned.drop(columns=['loan_amnt'], inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)
numerical_columns = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()

In [ ]:
# Remove unrealistic person_emp_exp values
original_row_count = df_cleaned.shape[0]
df_cleaned = df_cleaned[df_cleaned['person_emp_exp'] <= 70]
removed_count = original_row_count - df_cleaned.shape[0]
changelog.append({
    'column': 'person_emp_exp',
    'change': f'Removed {removed_count} row(s) where person_emp_exp > 70',
    'rationale': 'Unrealistic employment experience; values above 70 years are likely errors or outliers.',
}) 

In [ ]:
# Check cleaned data
df_cleaned.info()
df_cleaned.describe()

# Plot boxplot for cleaned data
plt.figure(figsize=(12, 8))
sns.boxplot(data=df_cleaned[numerical_columns])
plt.title('Boxplot of Numerical Features After Cleaning')
plt.xticks(rotation=45)
plt.show()

In [ ]:
for col in numerical_columns:
    fig = plt.figure(figsize=(8, 4))
    sns.histplot(df_cleaned[col], kde=True)
    plt.title(f"Distribution of {col}")
    plt.show()


In [ ]:
# Change summary intialization and check
change_summary = pd.DataFrame(changelog)
pd.set_option('display.max_colwidth', None)
display(change_summary)  

In [ ]:
# Check cleaned data
df_cleaned.info()
df_cleaned.describe()

# Plot boxplot for cleaned data
plt.figure(figsize=(12, 8))
sns.boxplot(data=df_cleaned[numerical_columns])
plt.title('Boxplot of Numerical Features After Cleaning')
plt.xticks(rotation=45)
plt.show()

# Feature Engineering

<h5>  Regroup the Person Education </h5>
Since *Master* and *Doctorate* degrees are both Graduate degrees, we can combine those two categories into one and rename as *Graduate*.

In [ ]:
# Create the new mappings
grad_edu = {'Master': 'Graduate',
            'Doctorate': 'Graduate'
}

#Create new field with the updated values
df_cleaned['person_education_new'] = df['person_education'].replace(grad_edu)

#View the new distribution for education
fig = plt.figure(figsize=(8, 4))
df_cleaned['person_education_new'].value_counts().plot(kind='bar')
plt.title(f"Distribution of Education")
plt.show()

#Drop the original field
df_cleaned.drop('person_education', axis=1, inplace=True)

#Update the change log
changelog.append({
    'column': 'person_education',
    'change': 'Combined Master and Doctorate categories into one group -- Graduate',
    'rationale': 'Doctorate had the lowest frequency out of all the categories which could impact analysis',
})

#Create dataframe to hold the subset of data
other_home_ownership = df[df['person_home_ownership'] == 'OTHER']
#Find the number of records that fall into this subset
print("Number of records: ", len(other_home_ownership))

other_home_ownership.describe(include='all')


#Look at the distribution for age
sns.histplot(other_home_ownership['person_age'], bins=10)

#Look at the distribution for loan amount
fig = plt.figure(figsize=(10, 4))
other_home_ownership['loan_status'].value_counts().plot(kind='bar')
plt.title(f"Distribution of Loan Status")
plt.show()



Within this subset of data, there is a difference in classes such that these records should remain in the dataset and the low frequency of the category should be handled so that it doesn't impact the analysis of the variable.

Since the *Rent* and *Mortgage* categories have high frequencies and *Own* and *Other* have lower frequencies, we can combine them into two groups *Homeowner* and *Non-Homeowner*. *Rent* and *Other* would fall under *Non-Homeowner** and *Mortgage* and *Own* would fall until *Homeowner*.

In [ ]:
#Mapping for new categories
ownership = {'RENT': 'Non-Homeowner',
             'OTHER': 'Non-Homeowner',
             'MORTGAGE': 'Homeowner',
             'OWN': 'Homeowner'
}

#Create new field with the updated values
df_cleaned['person_home_ownership_new'] = df['person_home_ownership'].replace(ownership)

#View the new distribution
fig = plt.figure(figsize=(8, 4))
df_cleaned['person_home_ownership_new'].value_counts().plot(kind='bar')
plt.title(f"Distribution of Home Ownership")
plt.show()

#Drop the original field
df_cleaned.drop('person_home_ownership', axis=1, inplace=True)


In [ ]:
#Update the change log
changelog.append({
    'column': 'person_home_ownership',
    'change': 'Combined RENT and OTHER into Non-Homeowner category and MORTGAGE and OWN into Homeowner category',
    'rationale': 'RENT and MORTGAGE had high frequencies while OWN and OTHER had low frequencies which could impact analysis',
})

In [ ]:
# Reilitialize categorical columns
categorical_columns = df_cleaned.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns:", categorical_columns)

In [ ]:
# Apply one-hot encoding to existing categorical_columns
df_cleaned = pd.get_dummies(df_cleaned, columns=categorical_columns, drop_first=True, dtype=int)

# Review structure of df_cleaned after encoding
df_cleaned.info()

# Add to changelog
changelog.append({
    'column': categorical_columns,
    'change': 'Applied one-hot encoding (drop_first=True)',
    'rationale': 'Converted categorical variables into binary indicators for model compatibility.',
})


In [ ]:
# Readd target variable
df_cleaned['loan_status'] = df['loan_status']

# Changelog for readding target variable
changelog.append({
    'column': 'loan_status',
    'change': 'Readded loan_status column',
    'rationale': 'Target variable for classification; readded after cleaning and preprocessing.',
})

In [ ]:
# Change summary intialization and check
change_summary = pd.DataFrame(changelog)
pd.set_option('display.max_colwidth', None)
display(change_summary)  

In [ ]:
# Export Change Summary to CSV
change_summary.to_csv("../Changelogs/change_summary.csv", index=False)

In [ ]:
# Export cleaned data
df_cleaned.to_csv("../Data/cleaned_loan_data.csv", index=False)
print("Cleaned data exported to cleaned_loan_data.csv")